In [1]:
from erddapy import ERDDAP
from erddapy import utilities

In [2]:
import json
import urllib
import requests
file = urllib.request.urlopen('https://raw.githubusercontent.com/IrishMarineInstitute/search-erddaps/master/erddaps.json')
servers = json.loads(file.read())
file.getcode()

200

In [3]:
servers=servers[:1]

In [4]:
print(servers)

[{'url': 'https://coastwatch.pfeg.noaa.gov/erddap/'}]


In [5]:
time_min = '2010-07-10T00:00:00Z'
time_max = '2016-08-10T00:00:00Z'
bbox = [-72.0, -69, 38, 41]

kw = {
   'search_for': 'all',
   'min_lon': bbox[0],
   'max_lon': bbox[1],
   'min_lat': bbox[2],
   'max_lat': bbox[3],
   'min_time': time_min,
   'max_time': time_max,
}

In [6]:
import pandas as pd
all_datasets=pd.DataFrame()

for key in servers:
    url = key['url']
    url = url.rstrip("/")
    r = requests.get(url)
    try:
        r.raise_for_status()
        e = ERDDAP(
                 server=url,
                 protocol='tabledap',
                 response='csv'
           )
        print(e.get_search_url(**kw))
    except requests.exceptions.RequestException as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.HTTPError as errh:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.ConnectionError as errc:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.Timeout as errt:
        print("Bad ERDDAP!!! {}".format(url))
        
    ds = pd.read_csv('%s'%e.get_search_url(**kw))
    ds['server'] = url
    
    datasets = ds[['server','Dataset ID','tabledap']]
    
    datasets.dropna(subset=['tabledap'],inplace=True)
    
    all_datasets = pd.concat([all_datasets,datasets])

https://coastwatch.pfeg.noaa.gov/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
all_datasets

In [ ]:
#tabledap = all_datasets.loc[all_datasets['tabledap'].notnull(),'Info']
#ii=0
#varnames=pd.DataFrame()
#varnames = []
for i in range(2):#all_datasets.shape[0]):
    dataset_url = '%s/tabledap/%s.csvp?latitude,longitude,time&distinct()' % (all_datasets['server'].iloc[int(i)],all_datasets['Dataset ID'].iloc[int(i)])
    coords = pd.read_csv(dataset_url)
    print(coords)
    #varnames.extend(meta['Variable Name'].unique())
    #varnames = pd.merge(varnames,meta.loc[meta['Attribute Name'] == "long_name",['Variable Name','Value']])

    #e = ERDDAP(
    #    server=all_datasets['server'].iloc[ii],
    #    protocol='tabledap')

    #e.dataset_id = all_datasets['Dataset ID'].unique()
    #e.protocol = "tabledap"
    #ii+=1
#print(varnames)    

        latitude (degrees_north)  longitude (degrees_east)  \
0                     -12.519775                154.414450   
1                     -12.509975                154.452650   
2                     -12.507925                154.467025   
3                     -12.500825                154.486075   
4                     -12.499300                154.469275   
5                     -12.499025                154.498275   
6                     -12.498950                154.519175   
7                     -12.498325                154.528225   
8                     -12.493425                154.503150   
9                     -12.491875                154.607825   
10                    -12.490350                154.548125   
11                    -12.490275                154.474450   
12                    -12.490150                154.512075   
13                    -12.487200                154.557075   
14                    -12.486350                154.502825   
15      

In [ ]:
all_datasets['Dataset ID'].iloc[int(i)]

In [ ]:
all_datasets['server'].iloc[0]

In [ ]:
# from erddapy import ERDDAP


# e = ERDDAP(
#   server='https://data.ioos.us/gliders/erddap',
#   protocol='tabledap',
# )

# e.response = 'csv'
# e.dataset_id = 'whoi_406-20160902T1700'
# e.constraints = {
#     'time>=': '2016-07-10T00:00:00Z',
#     'time<=': '2017-02-10T00:00:00Z',
#     'latitude>=': 38.0,
#     'latitude<=': 41.0,
#     'longitude>=': -72.0,
#     'longitude<=': -69.0,
# }
# e.variables = [
#     'depth',
#     'latitude',
#     'longitude',
#     'salinity',
#     'temperature',
#     'time',
# ]

In [ ]:
#     if url != 'http://erddap.emodnet-physics.eu/erddap' and \
#     url != 'https://erddap.marine.ie/erddap' and \
#     url != 'http://oos.soest.hawaii.edu/erddap' and \
#     url != 'http://erddap.secoora.org/erddap' and \
#     url != 'https://ecowatch.ncddc.noaa.gov/erddap' and \
#     url != 'http://dap.onc.uvic.ca/erddap':

In [ ]:
# laturl = 'http://erddap.bco-dmo.org/erddap/categorize/variableName/latitude/index.csv'
# lonurl = 'http://erddap.bco-dmo.org/erddap/categorize/variableName/longitude/index.csv'

# df_lat = pd.read_csv(laturl, header=[0])
# df_lon = pd.read_csv(lonurl, header=[0])

# df_datasets = pd.merge(df_lat, df_lon, on='Dataset ID')
# df_final = pd.DataFrame(columns=['latitude (degrees_north)', 'longitude (degrees_east)', 'Dataset ID'])

# for did in df_datasets['Dataset ID']:
#     if did != 'bcodmo_dataset_739309': # remove this once we reload all the datasets again.
#         dataset_url = 'http://erddap.bco-dmo.org/erddap/tabledap/%s.csvp?latitude,longitude&distinct()' % did
#         df_data = pd.read_csv(dataset_url, header=0, usecols=['latitude (degrees_north)', 'longitude (degrees_east)'])
#         df_data['Dataset ID'] = did
#         df_final = pd.concat([df_final, df_data], sort=False, ignore_index=True)
#         print(df_final.shape)

# lon = df_final['longitude (degrees_east)'].values
# lat = df_final['latitude (degrees_north)'].values